In [1]:
from confluent_kafka import Consumer, KafkaError
import psycopg2
import json

# Configuration du consommateur Kafka
conf = {
    'bootstrap.servers': 'host.docker.internal:9092',  # Adresse de votre broker Kafka
    'group.id': 'my_group',                           # Identifiant du groupe de consommateurs
    'auto.offset.reset': 'earliest'                   # Lire depuis le début si aucun offset n'existe
}

# Création du consommateur Kafka
consumer = Consumer(conf)

# Connexion à PostgreSQL
pg_conf = {
    'host': 'postgres',  # ou 'localhost' si vous ne l'exécutez pas dans Docker
    'port': '5432',
    'database': 'twitterdb',
    'user': 'fadi',
    'password': 'fadi'
}

try:
    # Établir une connexion à PostgreSQL
    print("Connexion à PostgreSQL...")
    conn = psycopg2.connect(**pg_conf)
    cursor = conn.cursor()

    # Créer une table pour les données brutes (Bronze)
    print("Création de la table bronze_mastodon si elle n'existe pas déjà...")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS bronze_mastodon (
        id BIGINT PRIMARY KEY,
        content TEXT,
        username TEXT,
        reblogs_count BIGINT,
        favourites_count BIGINT,
        replies_count BIGINT,
        language TEXT,
        url TEXT,
        media_url TEXT,
        media_preview_url TEXT,
        hashtags TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)
    conn.commit()
    print("Table bronze_mastodon prête.")

    # Souscription au topic Kafka
    consumer.subscribe(['mastodon_stream'])
    print("Souscription au topic 'mastodon_stream' réussie.")

    while True:
        # Poll pour les nouveaux contenus
        print("Polling Kafka pour de nouveaux contenus...")
        msg = consumer.poll(1.0)  # Attendre jusqu'à 1 seconde pour un contenu

        if msg is None:
            continue
        if msg.error():
            print(f"Erreur Kafka: {msg.error()}")
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Erreur irrécupérable: {msg.error()}")
                break

        try:
            # Décoder le contenu reçu de Kafka
            content_value = msg.value().decode('utf-8')
            print(f"Contenu reçu de Kafka : {content_value}")
            content_json = json.loads(content_value)

            # Récupérer les données importantes du contenu
            content_id = content_json.get('id')
            content_created_at = content_json.get('created_at')
            content = content_json.get('content')
            username = content_json.get('username')
            reblogs_count = content_json.get('reblogs_count', 0)
            favourites_count = content_json.get('favourites_count', 0)
            replies_count = content_json.get('replies_count', 0)
            language = content_json.get('language')
            url = content_json.get('url')

            # Récupérer les médias s'ils sont présents
            media_url = None
            media_preview_url = None
            if 'media_attachments' in content_json and content_json['media_attachments']:
                media_url = content_json['media_attachments'][0].get('url')
                media_preview_url = content_json['media_attachments'][0].get('preview_url')

            # Récupérer les hashtags s'ils sont présents (sous forme de chaîne)
            hashtags = content_json.get('hashtags', '')  # On récupère directement comme chaîne

            # Afficher le contenu reçu
            print(f"ID: {content_id}, Content: {content}, Hashtags: {hashtags}")

            # Vérifier si l'identifiant existe déjà dans la base de données
            cursor.execute("""
            SELECT 1 FROM bronze_mastodon WHERE id = %s
            """, (content_id,))
            exists = cursor.fetchone()

            if not exists:
                # Insérer le contenu dans la table bronze
                cursor.execute("""
                INSERT INTO bronze_mastodon (id, created_at, content, username, reblogs_count, favourites_count, replies_count, language, url, media_url, media_preview_url, hashtags) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """, (content_id, content_created_at, content, username, reblogs_count, favourites_count, replies_count, language, url, media_url, media_preview_url, hashtags))
                conn.commit()
                print(f"Contenu inséré dans la base de données : {content}")
            else:
                # Mettre à jour les colonnes counts et autres informations si le contenu existe déjà
                cursor.execute("""
                UPDATE bronze_mastodon 
                SET created_at = %s, content = %s, username = %s, reblogs_count = %s, favourites_count = %s, replies_count = %s, language = %s, url = %s, media_url = %s, media_preview_url = %s, hashtags = %s
                WHERE id = %s;
                """, (content_created_at, content, username, reblogs_count, favourites_count, replies_count, language, url, media_url, media_preview_url, hashtags, content_id))
                conn.commit()
                print(f"Contenu mis à jour dans la base de données : {content}")

        except json.JSONDecodeError as e:
            print(f"Erreur lors du décodage du contenu JSON : {e}")
        except Exception as e:
            print(f"Erreur inattendue : {e}")

except KeyboardInterrupt:
    print("Interruption par l'utilisateur.")

except Exception as e:
    print(f"Erreur générale : {e}")

finally:
    # Fermer les connexions
    print("Fermeture des connexions...")
    consumer.close()
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Connexions fermées.")

Connexion à PostgreSQL...
Création de la table bronze_mastodon si elle n'existe pas déjà...
Table bronze_mastodon prête.
Souscription au topic 'mastodon_stream' réussie.
Polling Kafka pour de nouveaux contenus...
Contenu reçu de Kafka : {"id": 113262789591954891, "created_at": "2024-10-06 22:16:55.092000+00:00", "content": "<p><span class=\"h-card\" translate=\"no\"><a href=\"https://mastodon.social/@UnicornRiot\" class=\"u-url mention\">@<span>UnicornRiot</span></a></span> </p><p><a href=\"https://itsgoingdown.org/dispatches-from-firestorm/\" target=\"_blank\" rel=\"nofollow noopener noreferrer\" translate=\"no\"><span class=\"invisible\">https://</span><span class=\"ellipsis\">itsgoingdown.org/dispatches-fr</span><span class=\"invisible\">om-firestorm/</span></a></p><p><a href=\"https://mastodon.social/tags/Helene\" class=\"mention hashtag\" rel=\"tag\">#<span>Helene</span></a> <a href=\"https://mastodon.social/tags/HurricaneHelene\" class=\"mention hashtag\" rel=\"tag\">#<span>Hurri